In [ ]:
E:\Study\Vision\project\Deep-MIML-Network>python feat_extractor.py
D:\Anaconda3\lib\site-packages\torchvision\models\densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)
Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to C:\Users\DL/.torch\models\densenet161-8d451a50.pth
100.0%
Traceback (most recent call last):
  File "feat_extractor.py", line 102, in <module>
    main()
  File "feat_extractor.py", line 93, in main
    imlist=os.listdir(imdir)
FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'dataset/images/solo/acoustic_guitar/1'

In [3]:
import os
#import h5py
#import json
import numpy as np
#import random
#import scipy.io.wavfile
#from scipy.io.wavfile import read
#from scipy import signal
#from sklearn.decomposition import NMF

In [2]:
#indexes1 = ['accordion', 'acoustic_guitar', 'banjo', 'cello', 'drum', 'electric_guitar', 'flute', 'french_horn', 'harmonica', 'harp', 'marimba', 'piano', 'saxophone', 'trombone', 'violin','xylophone']
#indexes2 = [[401], [402], [420], [486], [541], [546], [558], [566], [593], [594], [642], [579,881], [776], [875], [889],[642]]
#label = { k:v[0] for k,v in zip(indexes1,indexes2)}
#print(label)
indexes1 = ['accordion', 'acoustic_guitar', 'cello',  'flute',  'saxophone', 'trumpet', 'violin', 'xylophone']
indexes2 = [[401], [402], [486], [558], [776], [513], [889], [642]]
indexes3 = [[401], [402,546], [486], [558], [776], [513], [889], [642]]

## preprocess dataset
1. compute labels of images using ResNet101, and save the results as `*_labels.npy` files
2. compute NMF and save the result as `*_bases.npy` files
3. filepaths of `*_labels.npy`&`*_bases.npy` are stored in `train.h5`,`val.h5` and `test.h5`

In [ ]:
#from ResNet import resnet101_keras
from ResNet.resnet101_keras import resnet101_model, resnet_predict
#from ResNet.resnet152 import ResNet152, resnet152_predict
# Use pre-trained weights for Tensorflow backend
weights_path = 'ResNet/weights/ResNet-101-model.keras.h5'  #weights_path = 'weights/resnet101_weights_tf.h5'

# Test pretrained model - Inference mode
model = resnet101_model(weights_path)

d:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
#from dataset.NMF import *
from dataset.NMF import audio_import

path = 'h:/Study/bpfile/dataset/audios/'
#dir_list = os.listdir(path)
#dataset_num = 0   # 所有数据集个数，包括训练集，验证集
wav_files = []
bases_filepath = []
labels_filepath = []
label = np.zeros(1000)
flag=True

# 遍历数据集中所有的 wav 文件，记录他们的路径
for root,dirs,files in os.walk(path, topdown=True):
    for name in files:
        if '.wav' in name:
            wav_files.append(os.path.normcase(root+os.path.sep+name).replace('\\','/'))
            print(wav_files[-1])
'''# 对所有的 wav 文件计算 NMF
print('computing NMF ...')
nmf_base = audio_import.nmf(wav_files)
print('NMF computed!')
print(nmf_base.shape)'''

for i, fname in enumerate(wav_files):
    # 每个 wav 对应的 NMF 结果保存为 *.npy 文件，对应路径存入 bases
    nmf_base = audio_import.nmf([fname])
    np.save(fname[:-4]+'_bases'+'.npy', nmf_base[0])
    bases_filepath.append(fname[:-4]+'_bases'+'.npy')
    
    # 根据文件路径名确定当前音频中乐器的 label，作为 ground truth 训练
    for instrument, index in zip(indexes1, indexes2):
        if instrument in fname:
            label[index] = 1
    np.save(fname[:-4]+'_labels'+'.npy', label)
    labels_filepath.append(fname[:-4]+'_labels'+'.npy')

    print(bases_filepath[-1])
    print(labels_filepath[-1])
    print(np.array(nmf_base[0]).shape)
    
    
print(bases_filepath)
print(labels_filepath)

h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/1.wav
h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/10.wav
h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/11.wav
h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/12.wav
h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/13.wav
h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/14.wav
h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/15.wav
h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/16.wav
h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/17.wav
h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/18.wav
h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/19.wav
h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/2.wav
h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/20.wav
h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/21.wav
h:/study/bpfile/dataset/audios/duet/acoustic_guitarviolin/22.wav
h:/study/bpfile/dataset/aud

KeyboardInterrupt: 

## generate `train.h5` and `val.h5`

In [4]:
import h5py
import random

dataset_num = len(wav_files)
train_num = int(0.7*dataset_num)

# 随机选取 70% 的数据作为验证集
train_bases = random.sample(range(dataset_num), train_num)

# 路径 utf-8 编码，否则报错
train_bases_encode = []
train_labels_encode = []
val_bases_encode = []
val_labels_encode = []
for i in range(dataset_num):
    if i in train_bases:
        train_bases_encode.append(bases_filepath[i].encode(encoding='utf-8', errors='strict'))
        train_labels_encode.append(labels_filepath[i].encode(encoding='utf-8', errors='strict'))
    else:
        val_bases_encode.append(bases_filepath[i].encode(encoding='utf-8', errors='strict'))
        val_labels_encode.append(labels_filepath[i].encode(encoding='utf-8', errors='strict'))

h5f = h5py.File('dataset/train.h5', 'w')
h5f.create_dataset('bases', data=train_bases_encode)
h5f.create_dataset('labels', data=train_labels_encode)
h5f.close()

h5f = h5py.File('dataset/val.h5', 'w')
h5f.create_dataset('bases', data=val_bases_encode)
h5f.create_dataset('labels', data=val_labels_encode)
h5f.close()

d:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## generate `test.h5`

In [1]:
#from ResNet import resnet101_keras
from ResNet.resnet101_keras import resnet101_model, resnet_predict

# Use pre-trained weights for Tensorflow backend
weights_path = 'ResNet/weights/ResNet-101-model.keras.h5'  #weights_path = 'weights/resnet101_weights_tf.h5'
# Test pretrained model - Inference mode
model = resnet101_model(weights_path)

import json
with open('./labels.json','r') as f:
    ImageNet_Labels = json.load(f)     # keys should be tranformed to string

d:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
indexes1 = ['accordion', 'acoustic_guitar', 'cello',  'flute',  'saxophone', 'trumpet', 'violin', 'xylophone']
indexes2 = [[401], [402], [486], [558], [776], [513], [889], [642]]
indexes3 = [[401], [402,546], [486], [558], [776], [513], [889], [642]]

1. 随机取`20`张图片：错`11`个
2. 随机取`50`张图片：错`8`个

错的离谱的：
1. cello_2_flute_2
2. flute_2_saxphone_1

In [15]:
import os
import random
import numpy as np
#from dataset.NMF import *
from dataset.NMF import audio_import

test_path = 'h:/Study/bpfile/testset25/'
bases_filepath = []
labels_filepath = []

for cur_dir in os.listdir(test_path):
    if 'audio' in cur_dir:
        wav_dir = test_path+cur_dir+'/'
    if 'image' in cur_dir:
        img_dir = test_path+cur_dir+'/'

print(wav_dir)
print(img_dir)
        
wav_files = []
img_files = []

for f in os.listdir(wav_dir):   # 剔除非 wav 文件
    if '.wav' in f:
        #wav_files.append(wav_dir + f)
        wav_files.append(f)
for f in os.listdir(img_dir):   # 剔除目录下所有文件，只保留文件夹
    #print(f)
    #print(os.path.isdir(img_dir+f))
    if os.path.isdir(img_dir + f):
        #img_files.append(img_dir + f)
        img_files.append(f)
    
#print(wav_files)
#print(img_files)
#print('------------')
#print(wav_files.sort())
#print(img_files.sort())

wav_files.sort()
img_files.sort()

for wav_fname, img_folder in zip(wav_files, img_files):
    assert wav_fname[:-4] == img_folder    # 确保 wav 文件与图片文件夹相对应
    
    # NMF
    #nmf_base = audio_import.nmf([wav_dir+wav_fname])
    nmf_base = [1]
    #np.save(wav_dir+wav_fname[:-4]+'_bases'+'.npy', nmf_base[0])
    bases_filepath.append(wav_dir+wav_fname[:-4]+'_bases'+'.npy')
    
    # ResNet
    imgs = []
    softmax = []
    for img in os.listdir(img_dir+img_folder):
        if '.jpg' in img or '.png' in img:
            imgs.append(img)
    imgs = random.sample(imgs, 50)  # 由于图片数量过多，每个视频中只随机抽取 50 张图片进行预测
    for img in imgs:
        #print(resnet_predict(model, img_dir+img_folder+'/'+img).shape)
        softmax.append(list(resnet_predict(model, img_dir+img_folder+'/'+img)))
        #print(softmax.shape)
    softmax = np.sum(softmax, axis=0)
    #print(softmax.shape)
    #np.save(img_dir+img_folder+'/labels'+'.npy', softmax)
    labels_filepath.append(img_dir+img_folder+'/labels'+'.npy')
    
    print(bases_filepath[-1])
    print(labels_filepath[-1])
    
    '''prediction = np.argpartition(softmax, -5)[-5:]
    for i in range(5):
        print(prediction[i], ImageNet_Labels[str(prediction[i])], softmax[prediction[i]])
    print('------------')'''
    selected_label = np.zeros(8)
    for i,class_indexes in enumerate(indexes3):
        for index in class_indexes:
            selected_label[i] = selected_label[i] + softmax[index]
        selected_label[i] = selected_label[i] / len(class_indexes)
    prediction = np.argpartition(selected_label, -3)[-3:]
    #prediction = np.argsort(softmax)[-2:]
    for i in range(3):
        print(indexes2[prediction[i]], indexes1[prediction[i]], selected_label[prediction[i]])
    print('-----------------------------')
    #print(softmax)
    
#print(bases_filepath)
#print(labels_filepath)

h:/Study/bpfile/testset25/gt_audio/
h:/Study/bpfile/testset25/testimage/
h:/Study/bpfile/testset25/gt_audio/accordion_1_saxophone_1_bases.npy
h:/Study/bpfile/testset25/testimage/accordion_1_saxophone_1/labels.npy
[558] flute 0.003060430521145463
[402] acoustic_guitar 0.011718106135958806
[401] accordion 49.84189987182617
-----------------------------
h:/Study/bpfile/testset25/gt_audio/accordion_2_acoustic_guitar_2_bases.npy
h:/Study/bpfile/testset25/testimage/accordion_2_acoustic_guitar_2/labels.npy
[776] saxophone 0.22014552354812622
[401] accordion 0.6556233167648315
[402] acoustic_guitar 20.809664487838745
-----------------------------
h:/Study/bpfile/testset25/gt_audio/accordion_2_cello_3_bases.npy
h:/Study/bpfile/testset25/testimage/accordion_2_cello_3/labels.npy
[889] violin 0.49710944294929504
[486] cello 1.3811062574386597
[401] accordion 47.865562438964844
-----------------------------
h:/Study/bpfile/testset25/gt_audio/accordion_3_acoustic_guitar_3_bases.npy
h:/Study/bpfile/t

In [62]:
np.sum([[1,2],[3,4]],axis=0)

array([4, 6])

In [11]:
import h5py

test_bases_encode = []
test_labels_encode = []

for i in range(len(bases_filepath)):
    test_bases_encode.append(bases_filepath[i].encode(encoding='utf-8', errors='strict'))
    test_labels_encode.append(labels_filepath[i].encode(encoding='utf-8', errors='strict'))

h5f = h5py.File('dataset/test.h5', 'w')
h5f.create_dataset('bases', data=test_bases_encode)
h5f.create_dataset('labels', data=test_labels_encode)
h5f.close()

## read data in `train.h5`

In [ ]:
h5f = h5py.File('dataset/train.h5', 'r')
for name in h5f:
    print(name)
    for v in h5f[name]:
        print(v)
    
h5f.close()

In [10]:
h5f.close()

In [2]:
softmax = resnet_predict('ResNet/images/000001.jpg')

image size: (224, 224, 3)
batch_shape (1, 224, 224, 3)
softmax shape: (1, 1000)
Prediction ID: 401
Class: accordion, piano accordion, squeeze box
Score: 0.9954704



In [5]:
print(softmax.shape)

(1, 1000)


## to validate the gt labels extracted from file path are right

In [ ]:
import os

ff=[]
path = path = 'h:/Study/bpfile/dataset/audios/'
for root,dirs,files in os.walk(path, topdown=True):
    '''print(root)
    print('---')
    print(dirs)
    print('---')
    for name in files:
        print(name)
    print('--------------------------------------------')'''
    for name in files:
        ff.append(os.path.normcase(root+os.path.sep+name).replace('\\','/'))

for fff in ff:
    for instr in indexes1:
        print(fff)
        if instr in fff:
            print(instr+' yes')
        else:
            print(instr+' no')

## exercise for some python grammar

In [15]:
a = ['aa','cc','dd','ab','a1','h:abs','h4f','e:ss']
print(a)
print(a.sort())
print(a)

['aa', 'cc', 'dd', 'ab', 'a1', 'h:abs', 'h4f', 'e:ss']
None
['a1', 'aa', 'ab', 'cc', 'dd', 'e:ss', 'h4f', 'h:abs']


In [20]:
a = []
a.append([1,2])
a.append([7,9])
print(a)
np.sum(a,axis=0)

[[1, 2], [7, 9]]


array([ 8, 11])

In [12]:
import json

with open('./labels.json','r') as f:
    labels = json.load(f)
    
labels['401']

'accordion, piano accordion, squeeze box'

## generate dataset for AudioSet-SingleSource

In [ ]:
#f = open('./imagenet1000_clsid_to_human.json', encoding='utf-8')
#label = json.load(f)
#label = dict(zip(label.values(), label.keys()))
indexes1 = ['accordion', 'acoustic_guitar', 'banjo', 'cello', 'drum', 'electric_guitar', 'flute', 'french_horn', 'harmonica', 'harp', 'marimba', 'piano', 'saxophone', 'trombone', 'violin']
indexes2 = [[401], [402], [420], [486], [541], [546], [558], [566], [593], [594], [642], [579,881], [776], [875], [889]]
#indexes2 = [[401], [402], [420], [486], [541], [546], [558], [566], [593], [594], [642], [579], [776], [875], [889]]
#label = { k:v[0] for k,v in zip(indexes1,indexes2)}
#print(label)

In [ ]:
from ResNet.resnet101_keras import resnet101_model, resnet_predict

# Use pre-trained weights for Tensorflow backend
weights_path = 'ResNet/weights/ResNet-101-model.keras.h5'  #weights_path = 'weights/resnet101_weights_tf.h5'

# Test pretrained model - Inference mode
model = resnet101_model(weights_path)

In [ ]:
#from dataset.NMF import *
from dataset.NMF import audio_import
#from ResNet import resnet101_keras

path = 'e:/Study/Vision/project/Deep-MIML-Network/dataset/AudioSet-SingleSource/'
dir_list = os.listdir(path)
numOfTrain = 0
bases = []
labels = []
flag=True

for cur_dir in dir_list:
    #忽略非乐器数据
    if cur_dir not in indexes1:
        continue
        
    #读取wav文件
    for f in os.listdir(path+cur_dir):
        if '.wav' in f:
            fname = path+cur_dir+'/'+f
            break
    print([fname])
    
    #计算NMF并保存.npy文件
    if flag:
        nmf_base = audio_import.nmf([fname])[0]  #每个文件夹下只有一个wav文件
        print(nmf_base.shape)
        #flag=False
    np.save(path+cur_dir+'/'+f[5:-4]+'_bases'+'.npy', nmf_base)
    bases.append(path+cur_dir+'/'+f[5:-4]+'_bases'+'.npy')
    
    #读取当前文件夹下所有图片文件
    img_files = []
    #print(path+cur_dir)
    for root,dirs,files in os.walk(path+cur_dir, topdown=False):
        #print(root)
        for name in files:
            if '.png' in name or '.jpg' in name:
                img_files.append(os.path.normcase(root+os.path.sep+name).replace('\\','/'))
                #print(img_file[-1])
    #print(img_files)
    
    #用ResNet101检测图像中物体种类
    softmax = np.zeros(1000)
    for img in random.sample(img_files, len(img_files)//3):  #为了节约时间，随机抽取1/3的图片进行预测
        softmax = softmax + resnet_predict(model, img)[0]
    np.save(path+cur_dir+'/'+f[5:-4]+'_labels'+'.npy', softmax)
    labels.append(path+cur_dir+'/'+f[5:-4]+'_labels'+'.npy')
    
    numOfTrain = numOfTrain+1
    
    '''
    sample_rate, samples = scipy.io.wavfile.read(fname)
    frequencies, times, spectrogram = signal.spectrogram(samples, fs=sample_rate, nperseg=4410, noverlap=nperseg//2, nfft=2401)
    #print(sr_value)
    #print(x_value)'''
    
print(bases)
print(labels)

### generate `train.h5` and `val.h5`

In [ ]:
import h5py

bases_encode = []
labels_encode = []
for i,j in zip(bases,labels):
    bases_encode.append(i.encode(encoding='utf-8', errors='strict'))
    labels_encode.append(j.encode(encoding='utf-8', errors='strict'))


h5f = h5py.File('dataset/train.h5', 'w')
h5f.create_dataset('bases', data=bases_encode[0:numOfTrain])
h5f.create_dataset('labels', data=labels_encode[0:numOfTrain])
h5f.close()

h5f = h5py.File('dataset/val.h5', 'w')
h5f.create_dataset('bases', data=bases_encode[0:numOfTrain//2])
h5f.create_dataset('labels', data=labels_encode[0:numOfTrain//2])
h5f.close()

## generate `test.h5`

In [ ]:
import h5py

h5f = h5py.File('dataset/train.h5', 'r')
test_bases = []
test_labels = []

for name in h5f:
    print(name)
    if name == 'bases':
        for v in h5f[name]:
            test_bases.append(v)
    if name == 'labels':
        for v in h5f[name]:
            test_labels.append(v)
    
h5f.close()

#print(test_bases)
#print(test_labels)

In [ ]:
h5f.close()